## Proyecto Detección Enfermedades

### Integrantes de Grupo

### Jhosep Olivera
### Omar Huamani
### David Castro
### Lubel Ochoa

In [ ]:
# Instalar YOLOv8 (Ultralytics)
!pip install ultralytics

# Dependencias adicionales
#!pip install roboflow
!pip install opencv-python

In [ ]:
# Librerías necesarias
import os
import shutil
import yaml
import cv2
import numpy as np
import json
import matplotlib.pyplot as plt
#from roboflow import Roboflow
from ultralytics import YOLO
import torch  # Importar la librería torch

import random
from sklearn.model_selection import train_test_split

In [ ]:
from google.colab import drive

# Volver a montar Google Drive sin intentar eliminar el directorio
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import shutil
import os
from google.colab import drive

# Eliminar contenido de la carpeta /content/drive si existe
mount_point = '/content/drive'
if os.path.exists(mount_point):
    shutil.rmtree(mount_point)

# Volver a montar Google Drive
drive.mount(mount_point)


OSError: [Errno 125] Operation canceled: '/content/drive/.Encrypted/.shortcut-targets-by-id'

In [ ]:
BASE_DIR = '/content/drive/MyDrive/Colab Notebooks/Proyecto Final - Adv Deep Learnig/Plant Village'

DATA_ETIQUETADA = f'{BASE_DIR}/DATA_ETIQUETADA'
DATA_PROCESADA = f'{BASE_DIR}/DATA_PROCESADA'
METADATA = f'{BASE_DIR}/METADATA'


In [ ]:
categorias = {
    'Papa': {
        'Sana': 'papa_sana',
        'TTAR': 'papa_ttar',
        'TTEM': 'papa_ttem',
    },
    'Pimiento_Morron': {
        'Sano': 'pimiento_sano',
        'MBAC': 'pimiento_mbac',
    },
    'Tomate': {
        'ARAC': 'tomate_arac',
        'MFSEP': 'tomate_mfsep',
        'Sano': 'tomate_sano',
        'MBAC': 'tomate_mbac',
        'MFOL': 'tomate_mfol',
        'TTAR': 'tomate_ttar',
        'MDI': 'tomate_mdi',
        'VMO': 'tomate_vmo',
        'VRALH': 'tomate_vralh',
        'TTEM': 'tomate_ttem',
    }
}


In [ ]:
def convertir_labelme_agricultura(json_path, img_path, txt_path, clase_id):
    with open(json_path, 'r') as f:
        data = json.load(f)

    img = cv2.imread(img_path)
    if img is None:
        print(f"Imagen no encontrada: {img_path}")
        return

    h, w = img.shape[:2]
    lineas = []

    for shape in data['shapes']:
        puntos = np.array(shape['points'])
        if len(puntos) < 3:
            continue

        segment = []
        for x, y in puntos:
            segment.extend([f"{x/w:.6f}", f"{y/h:.6f}"])

        linea = f"{clase_id} " + " ".join(segment)
        lineas.append(linea)

    if lineas:
        with open(txt_path, 'w') as f:
            f.write("\n".join(lineas))


In [ ]:
def crear_estructura_yolo():
    splits = ['train','valid','test']
    tipos = ['images','labels']
    for s in splits:
        for t in tipos:
            os.makedirs(f"{DATA_PROCESADA}/{s}/{t}", exist_ok=True)

crear_estructura_yolo()


In [ ]:
import os

# Comprobar las carpetas creadas
for split in ['train', 'valid', 'test']:
    for tipo in ['images', 'labels']:
        folder_path = os.path.join(DATA_PROCESADA, split, tipo)
        if os.path.exists(folder_path):
            print(f"Carpeta {folder_path} existe")
        else:
            print(f"Carpeta {folder_path} NO existe")


Carpeta /content/drive/MyDrive/Colab Notebooks/Proyecto Final - Adv Deep Learnig/Plant Village/DATA_PROCESADA/train/images existe
Carpeta /content/drive/MyDrive/Colab Notebooks/Proyecto Final - Adv Deep Learnig/Plant Village/DATA_PROCESADA/train/labels existe
Carpeta /content/drive/MyDrive/Colab Notebooks/Proyecto Final - Adv Deep Learnig/Plant Village/DATA_PROCESADA/valid/images existe
Carpeta /content/drive/MyDrive/Colab Notebooks/Proyecto Final - Adv Deep Learnig/Plant Village/DATA_PROCESADA/valid/labels existe
Carpeta /content/drive/MyDrive/Colab Notebooks/Proyecto Final - Adv Deep Learnig/Plant Village/DATA_PROCESADA/test/images existe
Carpeta /content/drive/MyDrive/Colab Notebooks/Proyecto Final - Adv Deep Learnig/Plant Village/DATA_PROCESADA/test/labels existe


In [ ]:
clases_unificadas = {}
clase_index = 0

rutas = []

for cultivo, subclases in categorias.items():
    cultivo_path = os.path.join(DATA_ETIQUETADA, cultivo)

    for enfermedad, clase_nombre in subclases.items():
        clases_unificadas[clase_index] = clase_nombre
        carpeta = os.path.join(cultivo_path, enfermedad)
        rutas.append((carpeta, clase_index))
        clase_index += 1

print("CLASES CREADAS:")
for i, n in clases_unificadas.items():
    print(i, n)


CLASES CREADAS:
0 papa_sana
1 papa_ttar
2 papa_ttem
3 pimiento_sano
4 pimiento_mbac
5 tomate_arac
6 tomate_mfsep
7 tomate_sano
8 tomate_mbac
9 tomate_mfol
10 tomate_ttar
11 tomate_mdi
12 tomate_vmo
13 tomate_vralh
14 tomate_ttem


In [ ]:
def procesar_data():
    extensiones_img = ('.jpg', '.JPG', '.jpeg', '.JPEG', '.png', '.PNG')

    resumen_global = {
        "total_imagenes_detectadas": 0,
        "total_procesadas_nuevas": 0,
        "total_omitidas_ya_existentes": 0,
        "errores": [],
        "detalle_por_clase": {}
    }

    for carpeta, clase_id in rutas:
        clase_nombre = clases_unificadas[clase_id]
        resumen_global["detalle_por_clase"][clase_nombre] = {
            "train": {"nuevas": 0, "omitidas": 0},
            "valid": {"nuevas": 0, "omitidas": 0},
            "test":  {"nuevas": 0, "omitidas": 0}
        }

        imgs = [f for f in os.listdir(carpeta) if f.endswith(extensiones_img)]
        resumen_global["total_imagenes_detectadas"] += len(imgs)

        data_pares = []
        for img_name in imgs:
            base = os.path.splitext(img_name)[0]
            posibles_labels = [f"{base}.json", f"{base}.JSON", f"{base}.jason", f"{base}.JASON"]

            label_file = next(
                (lbl for lbl in posibles_labels if os.path.exists(os.path.join(carpeta, lbl))),
                None
            )

            if label_file:
                data_pares.append((img_name, label_file))
            else:
                resumen_global["errores"].append({
                    "imagen": img_name,
                    "motivo": "Label no encontrado",
                    "carpeta": carpeta
                })

        random.shuffle(data_pares)

        n = len(data_pares)
        t1, t2 = int(n * 0.7), int(n * 0.9)

        split = {
            "train": data_pares[:t1],
            "valid": data_pares[t1:t2],
            "test":  data_pares[t2:]
        }

        for s, contenido in split.items():
            for img_name, json_name in contenido:
                pref = clases_unificadas[clase_id]

                destino_img = f"{DATA_PROCESADA}/{s}/images/{pref}_{img_name}"
                destino_lbl = f"{DATA_PROCESADA}/{s}/labels/{pref}_{os.path.splitext(img_name)[0]}.txt"

                # 🔍 CONTROL DE PROCESAMIENTO PREVIO
                if os.path.exists(destino_img) and os.path.exists(destino_lbl):
                    resumen_global["total_omitidas_ya_existentes"] += 1
                    resumen_global["detalle_por_clase"][clase_nombre][s]["omitidas"] += 1
                    continue

                try:
                    shutil.copy2(os.path.join(carpeta, img_name), destino_img)
                    convertir_labelme_agricultura(
                        os.path.join(carpeta, json_name),
                        os.path.join(carpeta, img_name),
                        destino_lbl,
                        clase_id
                    )

                    resumen_global["total_procesadas_nuevas"] += 1
                    resumen_global["detalle_por_clase"][clase_nombre][s]["nuevas"] += 1

                except Exception as e:
                    resumen_global["errores"].append({
                        "imagen": img_name,
                        "error": str(e),
                        "split": s,
                        "clase": clase_nombre
                    })

    # ===========================
    # GUARDAR LOG DE PROCESAMIENTO
    # ===========================
    os.makedirs(METADATA, exist_ok=True)
    log_path = f"{METADATA}/procesamiento_log.json"

    with open(log_path, "w") as f:
        json.dump(resumen_global, f, indent=4)

    print("========================================")
    print("✔ PROCESAMIENTO COMPLETADO")
    print(f"✔ Imágenes detectadas: {resumen_global['total_imagenes_detectadas']}")
    print(f"✔ Nuevas procesadas: {resumen_global['total_procesadas_nuevas']}")
    print(f"✔ Omitidas (ya existían): {resumen_global['total_omitidas_ya_existentes']}")
    print(f"✔ Errores: {len(resumen_global['errores'])}")
    print(f"✔ Log guardado en: {log_path}")
    print("========================================")


In [ ]:
config_unificado = {
    'path': DATA_PROCESADA,
    'train': 'train/images',
    'val': 'valid/images',
    'test': 'test/images',
    'names': clases_unificadas,
    'nc': len(clases_unificadas)
}

with open(f"{DATA_PROCESADA}/dataset.yaml", 'w') as f:
    yaml.dump(config_unificado, f)


In [ ]:
os.makedirs(METADATA, exist_ok=True)

with open(f"{METADATA}/clases.json", 'w') as f:
    json.dump(clases_unificadas, f, indent=4)

print("METADATA generada correctamente.")


METADATA generada correctamente.


In [ ]:
# Verificar imágenes en train/images
train_images_path = "/content/drive/MyDrive/Colab Notebooks/Proyecto Final - Adv Deep Learnig/Plant Village/DATA_PROCESADA/train/images"
imagenes_train = [f for f in os.listdir(train_images_path) if f.lower().endswith(('.JPG', '.jpg', '.jpeg', '.png'))]
print(f"Imágenes encontradas en 'train/images': {imagenes_train}")

# Verificar etiquetas en train/labels
train_labels_path = "/content/drive/MyDrive/Colab Notebooks/Proyecto Final - Adv Deep Learnig/Plant Village/DATA_PROCESADA/train/labels"
etiquetas_train = [f for f in os.listdir(train_labels_path) if f.endswith('.txt')]
print(f"Etiquetas encontradas en 'train/labels': {etiquetas_train}")


Imágenes encontradas en 'train/images': ['papa_ttar_b2492a9e-c038-48a7-8524-8f0aca7dc870___RS_LB 4291.JPG', 'papa_ttar_d752ca70-320b-4d33-970f-8f3eead9141e___RS_LB 3166.JPG', 'papa_ttar_c4aec622-2e7b-49f6-bab3-6c4dce29f155___RS_LB 3118.JPG', 'papa_ttar_b167bd40-798f-4e70-a735-f191350dd057___RS_LB 4017.JPG', 'papa_ttar_bb3049f8-7377-4610-b13f-8daf6753bda7___RS_LB 4031.JPG', 'papa_ttar_b4e52186-de58-4c06-bbd2-2fb3dbeda13e___RS_LB 2655.JPG', 'papa_ttar_c63d5cca-9639-4d42-88c4-13572afa1dda___RS_LB 2747.JPG', 'papa_ttar_c970a188-e4e6-43cd-aff7-d0ac40d378dc___RS_LB 4834.JPG', 'papa_ttar_b574ad16-1c07-4f70-bf1d-e2314d2dd81f___RS_LB 2849.JPG', 'papa_ttar_b41284a3-27e9-4f0c-8946-4deb0b8fc238___RS_LB 2839.JPG', 'papa_ttar_bb1949c9-c779-4490-b1cc-d82302bc1df9___RS_LB 2901.JPG', 'papa_ttar_c0a69909-e94c-4155-a008-20b5445c56d7___RS_LB 3909.JPG', 'papa_ttar_b3e1ec14-6de4-4afc-9ceb-254469c8ce15___RS_LB 3910.JPG', 'papa_ttar_c7982d1f-7472-44ba-a39d-76085c4d90cd___RS_LB 3072.JPG', 'papa_ttar_d3811753-f

## Entrenamiento del Modelo y Guardado de la Metadata y Modelo

In [ ]:
import os
import json
from datetime import datetime
import shutil
import torch  # Asegúrate de que 'torch' esté importado
from ultralytics import YOLO

# Directorios
DATA_ETIQUETADA = "/content/drive/MyDrive/Colab Notebooks/Proyecto Final - Adv Deep Learnig/Plant Village/DATA_ETIQUETADA"
DATA_PROCESADA = "/content/drive/MyDrive/Colab Notebooks/Proyecto Final - Adv Deep Learnig/Plant Village/DATA_PROCESADA"
METADATA_DIR = "/content/drive/MyDrive/Colab Notebooks/Proyecto Final - Adv Deep Learnig/Plant Village/METADATA"
MODEL_PATH = "/content/drive/MyDrive/Colab Notebooks/Proyecto Final - Adv Deep Learnig/Plant Village/MODEL/best.pt"  # Ruta del modelo entrenado

# Función para guardar la metadata y el modelo entrenado
def guardar_metadata(dataset_stats, backup_path, destino, project_name):
    # Crear el diccionario con la información
    info = {
        "proyecto": "Plant Village",
        "fecha": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
        "estructura_principal": {
            "DATA_ETIQUETADA": os.listdir(DATA_ETIQUETADA),
            "DATA_PROCESADA": os.listdir(DATA_PROCESADA)
        },
        "estadisticas_dataset": dataset_stats,
        "backup_guardado_en": backup_path,
        "dataset_yaml": f"{DATA_PROCESADA}/dataset.yaml",
        "modelo_entrenado": MODEL_PATH,  # Ruta del modelo entrenado
        "project_name": project_name  # Guardamos el nombre del proyecto en la metadata
    }

    # Ruta para guardar la metadata
    metadata_path = f"{destino}/metadata.json"

    # Guardar la metadata en formato JSON
    with open(metadata_path, "w") as f:
        json.dump(info, f, indent=4)

    # Copiar el modelo entrenado al directorio de destino
    if os.path.exists(MODEL_PATH):
        modelo_destino = os.path.join(destino, "best.pt")
        shutil.copy(MODEL_PATH, modelo_destino)
        print(f"Modelo guardado en: {modelo_destino}")
    else:
        print(f"El modelo no se encuentra en la ruta especificada: {MODEL_PATH}")

    return metadata_path

# Función para configurar el entrenamiento y optimizar los recursos
def entrenar_modelo():
    # Verificar si hay GPU disponible en Colab
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    print(f"Usando {device} para el entrenamiento")

    model = YOLO('yolov8s-seg.pt')  # Asegúrate de que esta ruta sea correcta
    project_name = 'plant_disease_yolo'  # Nombre del proyecto

    # Configurar parámetros de entrenamiento para optimizar recursos
    model.train(
        data=f"{DATA_PROCESADA}/dataset.yaml",
        epochs=1,
        imgsz=416,  # Reducir el tamaño de las imágenes para ahorrar memoria
        batch=16,  # Reducir el tamaño del batch para no agotar la memoria
        device=device,  # Usar GPU si está disponible, si no, CPU
        name='agriculture_seg',
        project=project_name,  # Especificamos el nombre del proyecto en el entrenamiento
        cache='disk',  # Usar caché en disco para evitar agotar la RAM
        save=True,  # Guardar el modelo entrenado
        save_period=-1,  # No guardar el modelo intermedio, solo al final
        workers=4,  # Número de workers para cargar los datos en paralelo
        optimizer='AdamW',  # Usar un optimizador eficiente
        lr0=0.001,  # Aprendizaje inicial
        lrf=0.01,  # Tasa de aprendizaje final
        warmup_epochs=3,  # Calentamiento del modelo durante los primeros 3 epochs
        warmup_bias_lr=0.1,  # Ajustar el learning rate del sesgo
        momentum=0.937,  # Momento de optimización
        weight_decay=0.0005  # Decaimiento de los pesos
    )

    # Aquí puedes llamar a la función de guardar metadata después de entrenar
    dataset_stats = {"total_imagenes": 1000, "total_clases": 10}  # Aquí agregas las estadísticas de tu dataset
    backup_path = "/content/drive/MyDrive/Colab Notebooks/Proyecto Final - Adv Deep Learnig/Plant Village/backup"  # Ruta de backup
    metadata_guardada = guardar_metadata(dataset_stats, backup_path, METADATA_DIR, project_name)
    print(f"Metadata guardada en: {metadata_guardada}")

# Ejecutar el entrenamiento
entrenar_modelo()



Usando cpu para el entrenamiento
Ultralytics 8.3.237 🚀 Python-3.12.12 torch-2.9.0+cpu CPU (Intel Xeon CPU @ 2.20GHz)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=disk, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/drive/MyDrive/Colab Notebooks/Proyecto Final - Adv Deep Learnig/Plant Village/DATA_PROCESADA/dataset.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=1, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=416, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.001, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8s-seg.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=agriculture_seg6

In [ ]:
import os
import shutil
import json
from datetime import datetime

BASE = "/content/Plant Village"

DATA_ETIQUETADA = f"{BASE}/DATA_ETIQUETADA"
DATA_PROCESADA = f"{BASE}/DATA_PROCESADA"
METADATA = f"{BASE}/METADATA"

os.makedirs(METADATA, exist_ok=True)


In [ ]:
def analizar_dataset(data_procesada):
    resumen = {
        "train": {},
        "valid": {},
        "test": {}
    }

    for split in ["train", "valid", "test"]:
        img_dir = f"{data_procesada}/{split}/images"
        lbl_dir = f"{data_procesada}/{split}/labels"

        imgs = [f for f in os.listdir(img_dir) if f.endswith(('.jpg','.png','.jpeg'))]
        lbls = [f for f in os.listdir(lbl_dir) if f.endswith('.txt')]

        resumen[split]["num_imagenes"] = len(imgs)
        resumen[split]["num_labels"] = len(lbls)

    return resumen


In [ ]:
def backup_data_procesada(origen, destino):
    destino_final = os.path.join(destino, "DATA_PROCESADA_BACKUP")

    # si ya existe, borramos para actualizar
    if os.path.exists(destino_final):
        shutil.rmtree(destino_final)

    shutil.copytree(origen, destino_final)

    return destino_final


In [ ]:
def guardar_metadata(dataset_stats, backup_path, destino):
    info = {
        "proyecto": "Plant Village",
        "fecha": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
        "estructura_principal": {
            "DATA_ETIQUETADA": os.listdir(DATA_ETIQUETADA),
            "DATA_PROCESADA": os.listdir(DATA_PROCESADA)
        },
        "estadisticas_dataset": dataset_stats,
        "backup_guardado_en": backup_path,
        "dataset_yaml": f"{DATA_PROCESADA}/dataset.yaml"
    }

    metadata_path = f"{destino}/metadata.json"
    with open(metadata_path, "w") as f:
        json.dump(info, f, indent=4)

    return metadata_path


In [ ]:
import os
from collections import defaultdict

def analizar_dataset(data_procesada):
    """
    Analiza un dataset YOLO procesado sin asumir estructura rígida.
    Retorna estadísticas consolidadas.
    """

    stats = {
        "total_images": 0,
        "total_labels": 0,
        "splits": {},
        "clases": defaultdict(int),
        "errores": []
    }

    # Caso A: DATA_PROCESADA/{train,val,test}/{images,labels}
    estructura_A = os.path.isdir(os.path.join(data_procesada, "train"))

    # Caso B: DATA_PROCESADA/{images,labels}/{train,val,test}
    estructura_B = os.path.isdir(os.path.join(data_procesada, "images"))

    if estructura_A:
        splits = ["train", "val", "test"]
        for split in splits:
            img_dir = os.path.join(data_procesada, split, "images")
            lbl_dir = os.path.join(data_procesada, split, "labels")
            _analizar_split(img_dir, lbl_dir, split, stats)

    elif estructura_B:
        splits = ["train", "val", "test"]
        for split in splits:
            img_dir = os.path.join(data_procesada, "images", split)
            lbl_dir = os.path.join(data_procesada, "labels", split)
            _analizar_split(img_dir, lbl_dir, split, stats)

    else:
        raise RuntimeError(
            f"Estructura desconocida en DATA_PROCESADA: {data_procesada}"
        )

    return stats


def _analizar_split(img_dir, lbl_dir, split, stats):
    if not os.path.exists(img_dir) or not os.path.exists(lbl_dir):
        stats["errores"].append(f"{split}: directorios inexistentes")
        return

    imgs = [f for f in os.listdir(img_dir) if f.lower().endswith(('.jpg','.png','.jpeg'))]
    lbls = [f for f in os.listdir(lbl_dir) if f.endswith('.txt')]

    stats["splits"][split] = {
        "imagenes": len(imgs),
        "labels": len(lbls)
    }

    stats["total_images"] += len(imgs)
    stats["total_labels"] += len(lbls)

    # Conteo de clases desde labels
    for lbl in lbls:
        with open(os.path.join(lbl_dir, lbl)) as f:
            for line in f:
                if line.strip():
                    class_id = int(line.split()[0])
                    stats["clases"][class_id] += 1



In [ ]:
import os

imagenes = [
    f for f in os.listdir(TEST_DIR)
    if f.lower().endswith(('.jpg', '.png', '.jpeg'))
]

print(f"Imágenes de test encontradas: {len(imagenes)}")
print(imagenes[:10])


In [ ]:
import os

TEST_IMAGE = os.path.join(TEST_DIR, imagenes[0])

results = model.predict(
    source=TEST_IMAGE,
    conf=0.25,
    iou=0.5,
    save=True
)

print("✔ Inferencia realizada sobre:")
print(TEST_IMAGE)


In [ ]:
import os

base_dir = "/content/runs/segment"

print("Carpetas de predicción disponibles:")
for d in os.listdir(base_dir):
    if d.startswith("predict"):
        print(" -", d)

In [ ]:
import os

predict_dir = "/content/runs/segment/predict"

print("Archivos generados por YOLO:")
for f in os.listdir(predict_dir):
    print(" -", f)



In [ ]:
from IPython.display import Image, display
import os

predict_dir = "/content/runs/segment/predict"

imagenes = [
    f for f in os.listdir(predict_dir)
    if f.lower().endswith(('.jpg', '.png', '.jpeg'))
]

if not imagenes:
    raise RuntimeError("No se encontraron imágenes en la carpeta predict")

display(Image(filename=os.path.join(predict_dir, imagenes[0])))


In [ ]:
import os
import json
from datetime import datetime

TEST_DIR = "/content/drive/MyDrive/Colab Notebooks/Proyecto Final - Adv Deep Learnig/Plant Village/IMAGENES_DEMOSTRACION/TOMATE"
OUTPUT_DIR = "/content/drive/MyDrive/Colab Notebooks/Proyecto Final - Adv Deep Learnig/Plant Village/RESULTADOS_INFERENCIA"
METADATA_DIR = "/content/drive/MyDrive/Colab Notebooks/Proyecto Final - Adv Deep Learnig/Plant Village/METADATA"

os.makedirs(OUTPUT_DIR, exist_ok=True)
os.makedirs(METADATA_DIR, exist_ok=True)

metadata = {
    "modelo": MODEL_TRAINED_PATH,
    "timestamp": datetime.now().isoformat(),
    "imagenes": [],
    "resumen": {
        "total_imagenes": 0,
        "total_objetos_detectados": 0
    }
}

imagenes = [
    f for f in os.listdir(TEST_DIR)
    if f.lower().endswith(('.jpg', '.png', '.jpeg'))
]

if not imagenes:
    raise RuntimeError("No se encontraron imágenes en el directorio de test")

for img in imagenes:
    img_path = os.path.join(TEST_DIR, img)

    results = model.predict(
        source=img_path,
        save=True,
        project=OUTPUT_DIR,
        name="predicciones",
        exist_ok=True
    )

    r = results[0]
    n_obj = 0 if r.boxes is None else len(r.boxes)

    metadata["imagenes"].append({
        "imagen": img,
        "objetos_detectados": n_obj,
        "confianza_promedio": (
            float(r.boxes.conf.mean()) if r.boxes is not None else 0.0
        )
    })

    metadata["resumen"]["total_imagenes"] += 1
    metadata["resumen"]["total_objetos_detectados"] += n_obj

metadata_path = os.path.join(
    METADATA_DIR,
    f"inferencia_entrenada_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json"
)

with open(metadata_path, "w") as f:
    json.dump(metadata, f, indent=4)

print("=====================================")
print("✔ Inferencia con modelo entrenado completada")
print("✔ Resultados visuales en:", OUTPUT_DIR)
print("✔ Metadata en:", metadata_path)
print("=====================================")



In [ ]:
## DEMOSTRACIÓN CON VIDEO

In [ ]:
import os
import json
import cv2
from datetime import datetime
from IPython.display import display, Video
from ultralytics import YOLO

# Directorios donde se encuentran los videos y se guardarán los resultados
VIDEO_DIR = "/content/drive/MyDrive/Colab Notebooks/Proyecto Final - Adv Deep Learnig/Plant Village/VIDEO_DEMOSTRACION"
OUTPUT_DIR = "/content/drive/MyDrive/Colab Notebooks/Proyecto Final - Adv Deep Learnig/Plant Village/RESULTADOS_INFERENCIA"
METADATA_DIR = "/content/drive/MyDrive/Colab Notebooks/Proyecto Final - Adv Deep Learnig/Plant Village/METADATA"

# Asegurarse de que los directorios existan
os.makedirs(OUTPUT_DIR, exist_ok=True)
os.makedirs(METADATA_DIR, exist_ok=True)

# Ruta del modelo entrenado
MODEL_TRAINED_PATH = "/content/drive/MyDrive/Colab Notebooks/Proyecto Final - Adv Deep Learnig/Plant Village/METADATA/best.pt"

# Verificar si el modelo existe
if not os.path.exists(MODEL_TRAINED_PATH):
    raise FileNotFoundError(f"El modelo no se encuentra en la ruta especificada: {MODEL_TRAINED_PATH}")

# Cargar el modelo entrenado
model = YOLO(MODEL_TRAINED_PATH)

# Diccionario para almacenar los metadatos
metadata = {
    "modelo": MODEL_TRAINED_PATH,
    "timestamp": datetime.now().isoformat(),
    "videos": [],
    "resumen": {
        "total_videos": 0,
        "total_frames_procesados": 0,
        "total_objetos_detectados": 0
    }
}

# Obtener la lista de videos en el directorio
videos = [
    f for f in os.listdir(VIDEO_DIR)
    if f.lower().endswith(('.mp4', '.avi', '.mov', '.mkv'))
]

if not videos:
    raise RuntimeError("No se encontraron videos en el directorio de test")

# Procesar cada video
for video in videos:
    video_path = os.path.join(VIDEO_DIR, video)
    cap = cv2.VideoCapture(video_path)

    if not cap.isOpened():
        raise RuntimeError(f"No se pudo abrir el video: {video_path}")

    frame_count = 0
    total_objects_detected = 0
    output_video_path = os.path.join(OUTPUT_DIR, f"procesado_{video}")

    # Obtener el tamaño del primer frame para ajustar la resolución del video de salida
    ret, frame = cap.read()
    if not ret:
        raise RuntimeError(f"Error al leer el primer frame del video: {video_path}")

    height, width, _ = frame.shape
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    out = cv2.VideoWriter(output_video_path, fourcc, 30.0, (width, height))  # Ajusta según la resolución

    # Procesar el video frame por frame
    while True:
        ret, frame = cap.read()
        if not ret:
            break  # Si no hay más frames, terminar

        frame_count += 1

        # Aplicar el modelo sobre cada frame
        results = model.predict(source=frame)

        # Obtener el número de objetos detectados en el frame
        r = results[0]
        n_obj = 0 if r.boxes is None else len(r.boxes)
        total_objects_detected += n_obj

        # Escribir el frame procesado en el video de salida
        out.write(frame)

        # Almacenar los resultados del frame
        metadata["videos"].append({
            "video": video,
            "frame": frame_count,
            "objetos_detectados": n_obj,
            "confianza_promedio": float(r.boxes.conf.mean()) if r.boxes is not None else 0.0
        })

    cap.release()  # Liberar el video
    out.release()  # Liberar el video de salida

    # Actualizar los resúmenes
    metadata["resumen"]["total_videos"] += 1
    metadata["resumen"]["total_frames_procesados"] += frame_count
    metadata["resumen"]["total_objetos_detectados"] += total_objects_detected

# Guardar los metadatos en un archivo JSON
metadata_path = os.path.join(METADATA_DIR, f"inferencia_video_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json")
with open(metadata_path, "w") as f:
    json.dump(metadata, f, indent=4)

# Mostrar mensaje de finalización
print("=====================================")
print("✔ Inferencia con modelo entrenado completada")
print(f"✔ Video procesado guardado en: {OUTPUT_DIR}")
print(f"✔ Metadata guardado en: {metadata_path}")
print("=====================================")

# Mostrar el video procesado en la misma celda de Google Colab
display(Video(output_video_path, embed=True))


In [ ]:
import os

MODEL_TRAINED_PATH = "/content/drive/MyDrive/Colab Notebooks/Proyecto Final - Adv Deep Learnig/Plant Village/METADATA/best.pt"

# Comprobar si el archivo existe
if not os.path.exists(MODEL_TRAINED_PATH):
    print(f"El modelo no se encuentra en la ruta especificada: {MODEL_TRAINED_PATH}")
else:
    print(f"El modelo se encuentra en la ruta: {MODEL_TRAINED_PATH}")

